In [ ]:
! pip install --quiet \
  "langchain>=0.2" \
  "langgraph>=0.2" \
  faiss-cpu \
  openai \
  tiktoken \
  pydantic \
  typing-extensions \
  langchain_community

In [ ]:
! pip install langchain_openai

In [ ]:
import os

In [ ]:
import pandas as pd
from datasets import Dataset

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI

# ✅ OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = "fill out your API key"
openai_api_key = os.environ["OPENAI_API_KEY"]  # ✅ API 키를 직접 가져오기

# ✅ OpenAI 임베딩 모델 적용 (API 키 전달 방식 수정)
openai_embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",# ✅ 최신 OpenAI 한국어 지원 임베딩 모델
    openai_api_key=openai_api_key,  # ✅ API 키를 직접 인자로 전달
)

/tmp/ipython-input-1232688011.py:9: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  openai_embeddings = OpenAIEmbeddings(


In [ ]:
faiss_path = "/content/drive/MyDrive/person_total"

In [ ]:
! pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.2 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import FAISS

In [ ]:
db2 = FAISS.load_local(faiss_path, openai_embeddings, allow_dangerous_deserialization=True)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 프롬프트: {context}, {question} 변수명 일치
single_stuff_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=("""
        "당신은 개인정보 보호법에 정통한 법률 전문가입니다.\n"
        "주어진 문서만을 바탕으로 사용자가 제시한 질문에 대해 구체적인 답변과 해당 법률 조항을 명시해 주세요.\n"
        "답변을 제시할 때 두 개의 역할에서 도출된 답변을 적절하게 사용해 주세요.\n"
        "적절한 답변이 불가능할 시, 'I don't know'로 대답해 주세요.\n\n"
        "추상적 단어의 처리방법:\n"
        "- 비공개 장소 : 화장실, 개인 객실 등\n"
        "- 민감정보 : 주민등록번호, 주소, 연락처 등\n"
        "위와 같이 질문의 추상적인 단어는 네가 보유하고 있는 지식체계 내에서 구체화하여 고려한 다음 답변해 주세요.\n"
        "질문 속 지시대명사 간의 관계를 명확히 파악한 뒤, 답변 작성 시 이를 참고해 주세요.\n"
        "법 체계의 위계는 '개인정보 보호법'이 우선이며, 이를 바탕으로 '개인정보 보호법 시행령'이 집행됩니다.\n\n"
        "다음은 검색된 문서 내용입니다:\n\n"

--- 컨텍스트 시작 ---
{context}
--- 컨텍스트 끝 ---

[질문]
{question}

"""
)
)

# LLM
openai_llm = ChatOpenAI(
    model="gpt-4o-mini",        # 필요 시 "gpt-3.5-turbo"로 교체 가능
    temperature=0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# 리트리버
retriever = db2.as_retriever(search_type="similarity",
                             search_kwargs={"k": 4, "fetch_k": 10})

# RetrievalQA (stuff 체인) - document_variable_name 꼭 지정!
single_chain = RetrievalQA.from_chain_type(
    llm=openai_llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type="stuff",
    chain_type_kwargs={
        "prompt": single_stuff_prompt,
        "document_variable_name": "context",
    },
)

# 실행 함수 정의
def run_single(query: str):
    result = single_chain({"query": query})
    answer = result["result"]
    contexts = [doc.page_content for doc in result["source_documents"]]
    sources = result["source_documents"]
    return {
        "answer": answer,
        "contexts": contexts,
        "sources": sources
    }

In [ ]:
# 메인 실행부
if __name__ == "__main__":
    q = "사망자의 개인정보도 법적으로 보호되나요?"
    result = run_single(q)
    print("\n=== ANSWER ===\n", result["answer"])
    print(f"\n[contexts used: {len(result['contexts'])}]")

    # 소스 확인용 (선택)
    for i, d in enumerate(result["sources"], 1):
        print(f"\n--- SOURCE {i} ---\n", (d.page_content or "")[:800])